In [1]:
import pandas

In [2]:
df = pandas.read_csv('data/master_df.csv', index_col=0)

In [3]:
df.sample(5)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
179,Albany,689.0,239.0,29.0,1931.0,"FY 2017, 1st Q, Oct. 1-Dec. 31, 2016",New York,ALB,2016.0,FY 2017
135,Boston,4202.0,4699.0,517.0,9980.0,"FY 2018, 3rd Q, April 1-June 30, 2018",Massachusetts,BOS,2018.0,FY 2018
143,San Juan,66.0,1142.0,0.0,0.0,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",Puerto Rico,SAJ,NaN,FY 2020
179,Buffalo,1131.0,1125.0,163.0,2316.0,"FY 2016, 1st Q, Oct. 1-Dec.31, 2015",New York,BUF,2015.0,FY 2016
115,Wichita,436.0,243.0,46.0,1333.0,"FY 2018, 2nd Q, Jan. 1-March 31, 2018",Kansas,WIC,2018.0,FY 2018


In [4]:
code_to_city = {}
code_to_state = {}
city_to_code = {}
for i,entry in df.dropna(subset=['code']).iterrows():
#     print(entry.field_office, entry.code)
    code_to_city[entry.code.strip()] = entry.field_office.strip()
    code_to_state[entry.code.strip()] = entry.state.strip()
    city_to_code[entry.field_office.strip()] = entry.code.strip()
    # discovered a few missing entries
code_to_city['CSC'] = float('nan')
code_to_state['CSC'] = 'California'

code_to_city['NBC'] = 'National Benefits Center'
code_to_state['NBC'] = float('nan')

code_to_city['NSC'] = 'NEBRASKA SERVICE CENTER'
code_to_state['NSC'] = 'Nebraska'

code_to_city['VSC'] = 'VERMONT SERVICE CENTER'
code_to_state['VSC'] = 'Vermont'

code_to_city['TSC'] = 'TEXAS SERVICE CENTER'
code_to_state['TSC'] = 'Texas'

# don't know what this is, and it doesn't have a label
code_to_city['GCU'] = "Garden City"
code_to_state['GCU'] = "New York"

code_to_city['ANC'], code_to_state['ANC']

('Anchorage', 'Alaska')

In [5]:
print(len(df))
df.sample(3)

2963


,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
109,Wichita,304.0,435.0,14.0,496.0,"FY 2014, 4th Q, July 1-Sept. 30, 2014",Kansas,WIC,2014.0,FY 2014
145,St. Paul,3475.0,2166.0,276.0,16637.0,"FY 2018, 2nd Q, Jan. 1-March 31, 2018",Minnesota,SPM,2018.0,FY 2018
45,San Jose,4175.0,3578.0,273.0,6249.0,"FY 2015, 4th Q, July 1-Sept. 30, 2015",California,SNJ,2015.0,FY 2015


In [6]:
df.dtypes

field_office               object
Applications Received2    float64
Approved3                 float64
Denied4                   float64
Pending5                  float64
quarter                    object
state                      object
code                       object
year                      float64
fy                         object
dtype: object

In [7]:
sum_df = df.dropna(subset=['code']).groupby(['code', 'fy']).sum()
sum_df

Applications Received2  Approved3  Denied4  Pending5    year
code fy                                                                   
1    FY 2016                     7.0        8.0      0.0       0.0  2016.0
     FY 2017                     0.0        0.0      1.0       0.0  8067.0
     FY 2018                     0.0        0.0      0.0       0.0  8071.0
     FY 2019                     0.0        0.0      0.0       0.0  4037.0
14   FY 2016                    11.0        8.0      0.0       4.0  2016.0
...                              ...        ...      ...       ...     ...
YAK  FY 2016                  2033.0     1515.0    105.0    3059.0  8063.0
     FY 2017                  2280.0     1997.0    225.0    4453.0  8067.0
     FY 2018                  2042.0     2216.0    112.0    3102.0  8071.0
     FY 2019                   858.0     2747.0     60.0    1395.0  4037.0
     FY 2020                    33.0     1924.0      0.0       0.0     0.0

[985 rows x 5 columns]

In [8]:
del sum_df['year']

In [9]:
sum_df['field_office'] = sum_df.index.map(lambda tup: code_to_city[tup[0]])

In [10]:
sum_df['state'] = sum_df.index.map(lambda tup: code_to_state[tup[0]])

In [11]:
sum_df['nationality'] = sum_df.state.map(
    lambda state: 'international' if state in ['United Kingdom', 'Italy', 'Germany', 'South Korea']
                  else 'US (territory)' if state in ['Guam', 'U.S. Virgin Islands', 'Puerto Rico']
                  else 'US'
)

In [12]:
sum_df[sum_df.nationality == 'US (territory)'].to_csv('data/sum_by_year_territory.csv')
sum_df[sum_df.nationality == 'US'].sort_values(by=['state', 'field_office']).to_csv('data/sum_by_year.csv')
sum_df[sum_df.nationality == 'international'].to_csv('data/sum_by_year_international.csv')